**From the documentation**

http://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('./datasets/sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lr_model = lr.fit(training)

In [8]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lr_model.intercept

0.14228558260358093

In [11]:
training_summary = lr_model.summary

In [12]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

Split train and test

In [14]:
all_data = spark.read.format('libsvm').load('./datasets/sample_linear_regression_data.txt')

In [15]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [21]:
train_data.count(), test_data.count()

(341, 160)

In [22]:
correct_model = lr.fit(train_data)

In [25]:
test_results = correct_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -27.43785596399608|
|-26.674050930690253|
|-24.082371203716093|
| -22.08142339351675|
|-19.569317582220293|
|-20.066374004638263|
|-20.885929525673586|
| -19.47004560292366|
|-16.441458569986633|
|-19.922103526695587|
|-15.857252950317893|
| -14.17500041228667|
|-16.041943683969777|
|-13.262962200450605|
|-12.101586048396099|
|  -9.83991721075817|
|-12.210772550442055|
| -13.28748004831059|
|-10.211219212207203|
|-12.338335562709895|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

10.617060065948424

In [29]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [30]:
predictions = correct_model.transform(unlabeled_data)

In [31]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -1.133622905747348|
|(10,[0,1,2,3,4,5,...|-0.1314324977928179|
|(10,[0,1,2,3,4,5,...| 0.5714871133931222|
|(10,[0,1,2,3,4,5,...| 0.6490356293509456|
|(10,[0,1,2,3,4,5,...| -0.315243192053133|
|(10,[0,1,2,3,4,5,...| 0.3990553892665425|
|(10,[0,1,2,3,4,5,...| 2.0400070527750036|
|(10,[0,1,2,3,4,5,...| 1.1948320369190275|
|(10,[0,1,2,3,4,5,...|-0.8852621626893139|
|(10,[0,1,2,3,4,5,...|  3.229896505384482|
|(10,[0,1,2,3,4,5,...|0.13373733726932352|
|(10,[0,1,2,3,4,5,...|-1.2008573110256273|
|(10,[0,1,2,3,4,5,...| 0.9854607094273444|
|(10,[0,1,2,3,4,5,...| 0.2230341363459905|
|(10,[0,1,2,3,4,5,...| 0.1965991457209843|
|(10,[0,1,2,3,4,5,...|-2.0174331546712563|
|(10,[0,1,2,3,4,5,...| 0.5949972854264293|
|(10,[0,1,2,3,4,5,...| 1.9590641115328074|
|(10,[0,1,2,3,4,5,...|-0.7347004455757294|
|(10,[0,1,2,3,4,5,...| 1.7382052208008623|
+----------